In [ ]:
import os
from scripts.paper_figures.input import Input

import basty.project.experiment_processing as experiment_processing
from scripts.paper_figures.process_results import BehaviorData

from basty.utils import misc

In [ ]:

RESULTS_FOLDER = r'Z:\mfk\basty-projects\backup-allHaltere\results\semisupervised_pair_kNN\predictions.100NN.neighbor_weights-distance.neighbor_weights_norm-count.activation-standard.voting-soft.voting_weights-None\exports'

BODY_PART_SETS = {"HaltereSwitch": "prob", "Feeding": "prob"}

CONFIG_PATH = r"Z:\mfk\basty-projects\main_cfg.yaml"

PROJECT_PATH = os.path.dirname(CONFIG_PATH)

TMP_RESULT_FOLDER = os.path.join(PROJECT_PATH,'tmp_results')

project = experiment_processing.Project(CONFIG_PATH)

BEHAVIORS = ['HaltereSwitch']

BODY_PART_SETS = {"ProboscisPumping": "prob", "Feeding": "prob", "HaltereSwitch": "halt"}

In [ ]:
io_process = Input(project, RESULTS_FOLDER, TMP_RESULT_FOLDER)

In [ ]:
expt_info_df = io_process.load_expt_info()
df_data = io_process.load_predictions()

In [ ]:
# Drop Noise, Feeding, Grooming, ProboscisPumping from df_data
df_data.drop(['Noise','HaltereSwitch'], axis=1, inplace=True)

io_process.create_binary_masks_subfolders(BEHAVIORS)